# Training tree-based models for time series forecasting

After working with the ARIMA model yesterday, today you will use the same data to fit a Random Forest (RF) and XGBoost model on it.

Throughout the seminar, we will use the following splits for training, validation, and testing. Make sure to keep the tests unseen until the final evaluation (information leakage):

- Training set: 2009-2013
- Validation set: 2014
- Test set: 2015-2016

Required python packages: pandas, numpy, matplotlib, scikit-learn, xgboost

## Task 1: Feature Engineering

For random forest and xgboost, engineer features for the flu-trends dataset by incorporating lagged values (e.g., 8 weeks); ensure that your feature engineering process does not introduce information from future data points into past records.

> For instance, use the past 8 weeks to predict the next week.X-columns: FluVisits_t-8, FluVisits_t-7, ..., FluVisits_t-1
>
> You can also predict more than one week ahead. Then you need to shift the next samples accordingly.X-columns: FluVisits_t-8, FluVisits_t-7, ..., FluVisits_t-1
Y-columns: FluVisits_t, FluVisits_t+1, FluVisits_t+2

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import os


In [4]:
test = pd.read_csv(os.path.join('..', 'data', 'interim', 'test.csv'))
train = pd.read_csv(os.path.join('..', 'data', 'interim', 'training.csv'))
val = pd.read_csv(os.path.join('..', 'data', 'interim', 'validation.csv'))

In [6]:
print(test.shape)
print(train.shape)
print(val.shape)

(78, 132)
(236, 132)
(52, 132)


```python
 def create_features(data: pd.DataFrame, target_column: str, feature_length: int = 8, prediction_length: int = 2) -> [np.ndarray, np.ndarray]:
    ...
    return X, y
```

In [24]:
def create_features(data: pd.DataFrame, target_column: str, feature_length: int = 8, prediction_length: int = 2) -> tuple[np.ndarray, np.ndarray]:
    """
    Create features and target arrays for time series forecasting.

    Args:
        data (pd.DataFrame): The input data containing the time series.
        target_column (str): The name of the target column to predict.
        feature_length (int, optional): The number of past observations to use as features. Defaults to 8.
        prediction_length (int, optional): The number of future observations to predict. Defaults to 2.

    Returns:
        tuple[np.ndarray, np.ndarray]: The feature and target arrays.
    """
    
    target_data = data[target_column].values
    for i in range(feature_length, len(data) - prediction_length + 1): # Iterate over rows from e.g. 8 to len(data)-2
        features = target_data[i-feature_length:i] # Get the last 8 values as features as np array
        target = target_data[i:i+prediction_length] # Get the next 2 values as target as np array
        # for first iteration, initialize X and y
        if i == feature_length:
            X = np.array([features])
            y = np.array(target)
        else: # Add new rows to ndarrays
            X = np.vstack([X, features])
            y = np.vstack([y, target])
    return X, y

In [25]:
X, y = create_features(train, target_column="FluVisits", prediction_length=2, feature_length=8)
X.shape, y.shape

((227, 8), (227, 2))

## Task 2: Fitting the models

Task 2.1: Fit a Random Forest model to the data. Evaluate the model using the test set.

Task 2.2: Fit a XGBoost model to the data. Evaluate the model using the test set.

Describe the performance of the models and compare them in a few sentences. How do they perform in comparison to each other?

## Task 3: Hyperparameter tuning via grid search

Tune the hyperparameters of the Random Forest and XGBoost models using grid search. Focus on the following hyperparameters:

Random Forest:
- n_estimators
- max_depth
- ...

XGBoost:
- gamma
- max_depth
- eta
- ...

Optimize the hyperparameters using the validation set and the refit the models using the best hyperparameters on the entire training set and evaluate them on the test set.

## Task 4: Time series cross-validation

Implement a time series cross-validation strategy to optimize the hyperparameters of the models. Except for the cross-validation strategy, the procedure is the same as in Task 3. You only need to define different training and validation sets before performing the grid search. In the end, you take the parameters that performed best across all folds and refit the models on the entire training set and evaluate them on the test set.

# Optional Task:
- Take a look which other features could be good predictors for the flu visits. How does including additional features impact the overall model performance?